[Information Visualization Tutorials](https://infovis.fh-potsdam.de/tutorials/) · FH Potsdam · Summer 2020


# Tutorial 2: Data wrangling

Welcome back! This tutorial shows you a few tricks for preparing data for visualization. You will see how data can be loaded, parsed, and examined. For this we will continue to work with the **Pandas** package, in particular with the DataFrame data structure, and get to know a few additional helpers. 

*Just as in the first tutorial, you should be able to run the notebook yourself, edit the contents of all cells, and try things out. Here and there particular opportunities for such edits are highlighted with a pencil.*

In [ ]:
import pandas as pd

## Loading 

The first step is to bring the data into the purview of your notebook. So regardless of data structure and format, you need to have access to the data set. We will briefly cover four common ways of loading data into your Jupyter notebook.

### Enter data directly

The simplest way to add data to your notebook is to enter it verbatim into the notebook as we have seen with the capital cities in the first tutorial:

In [ ]:
cities = pd.DataFrame({
  "name": ["Athens", "Bratislava", "Copenhagen", "Dublin"],
  "area": [39, 367.6, 86.2, 115],
  "elevation": [170, 152, 14, 20],
  "population": [664046, 429564, 602481, 553165]
  }
)

cities

,name,area,elevation,population
0,Athens,39.0,170,664046
1,Bratislava,367.6,152,429564
2,Copenhagen,86.2,14,602481
3,Dublin,115.0,20,553165


✏️ *Add a column for years when you have visited or plan to visit these cities*

### Open a local file

You might also want to open a local file. Because this notebook is hosted on Colab, you can use a few example files in the `sample_data` directory that you can browser on the left side under the folder icon. 

We can open a file using Python's built-in `open()` method, after which we can `read()` its contents into the variable `anscombe` and finally `close()` it again. In this case the data is in the JSON format, which we will need to parse. We'll get to this later. You can open all kinds of formats. Here we know that we are dealing with a JSON file because of its extension.

(If you're running this notebook locally, you can download the `sample_data` directory from [here](http://infovis.fh-potsdam.de/tutorials/sample_data.zip). You need to unzip the directory at the same level where you placed this notebook, to do the following step).


In [ ]:
file = open("sample_data/anscombe.json")
anscombe_json = file.read()
file.close()

anscombe_json

'[\n  {"Series":"I", "X":10.0, "Y":8.04},\n  {"Series":"I", "X":8.0, "Y":6.95},\n  {"Series":"I", "X":13.0, "Y":7.58},\n  {"Series":"I", "X":9.0, "Y":8.81},\n  {"Series":"I", "X":11.0, "Y":8.33},\n  {"Series":"I", "X":14.0, "Y":9.96},\n  {"Series":"I", "X":6.0, "Y":7.24},\n  {"Series":"I", "X":4.0, "Y":4.26},\n  {"Series":"I", "X":12.0, "Y":10.84},\n  {"Series":"I", "X":7.0, "Y":4.81},\n  {"Series":"I", "X":5.0, "Y":5.68},\n\n  {"Series":"II", "X":10.0, "Y":9.14},\n  {"Series":"II", "X":8.0, "Y":8.14},\n  {"Series":"II", "X":13.0, "Y":8.74},\n  {"Series":"II", "X":9.0, "Y":8.77},\n  {"Series":"II", "X":11.0, "Y":9.26},\n  {"Series":"II", "X":14.0, "Y":8.10},\n  {"Series":"II", "X":6.0, "Y":6.13},\n  {"Series":"II", "X":4.0, "Y":3.10},\n  {"Series":"II", "X":12.0, "Y":9.13},\n  {"Series":"II", "X":7.0, "Y":7.26},\n  {"Series":"II", "X":5.0, "Y":4.74},\n\n  {"Series":"III", "X":10.0, "Y":7.46},\n  {"Series":"III", "X":8.0, "Y":6.77},\n  {"Series":"III", "X":13.0, "Y":12.74},\n  {"Series"

✏️ *Take a look into the sample_data directory and try open any other file*

### Get data via a URL

There are some methods that can directly load a dataset via a URL, i.e., a web address. For others you might have to retrieve the file first to continue parsing it. The `requests` package helps you to send HTTP requests and retrieve the responses. 

In the following, the news feed of Tagesschau is retrieved via an HTTP GET request. Note that the news feed is made available as an XML format; of course you can retrieve all kinds of file formats using this method:

In [ ]:
import requests

response = requests.get('https://www.tagesschau.de/xml/rss2/')
tagesschau_xml = response.text
tagesschau_xml[:100] # this displays the first 100 characters

'<?xml version="1.0" encoding="UTF-8" ?>\n<?xml-stylesheet href="/resources/xsl/rss2.jsp" type="text/x'

✏️ *Find the news feed for another webpage and try to load it*

### Use an API

Some web platforms require the use of an API (application programming interface) to get access to their data. Simply put, an API is a structured way to request and retrieve data. Oftentimes it is just a specific way to format the URL.

The German National Library offers an [API to query Entity Facts](https://www.dnb.de/EN/Professionell/Metadatendienste/Datenbezug/Entity-Facts/entityFacts_node.html) contained in the GND (Gemeinsame Normdatei). In this case the API provides the data in the JSON format, which has become quite common for web APIs, but you will also encounter many other formats. 

To retrieve information for a given GND entity by its id, such as the GND entry for the artist [Käthe Kollwitz](https://en.wikipedia.org/wiki/Käthe_Kollwitz) you have to put the `base_url` together with the `gnd_id`: 

In [ ]:
base_url = "https://hub.culturegraph.org/entityfacts/"
gnd_id = "118564943" # GND identifier from the Wikipedia page on Käthe Kollwitz
gnd_response = requests.get(base_url+gnd_id).text

gnd_response[:100]

'{\n  "@context" : "https://hub.culturegraph.org/entityfacts/context/v1/entityfacts.jsonld",\n  "@id" :'

✏️ *Prepare an API request with a GND id of another person of German history*

## Parsing 

Apart from our little cities example, so far we have only loaded the data into unstructured strings. To be able to analyze the data, we have to turn the unstructured strings of symbols into a practical data structure of the DataFrame that we can work with. This process is commonly referred to as ‘parsing’. 

As we have seen above, data can come in various file formats, which are in turn more or less appropriate for particular data structures. We'll cover four typical ones in the following section, but we will see more over the course of the tutorials to come.

The different ways of loading data (e.g., by file path or URL) are independent from the particular data formats provided. For example, you can load CSV data from a local file or from a web address. While the files typically indicate with the extension what format they have, URLs or APIs may not have these. If it is not clear, you may have to check the documentation or take a peek into the file.

### CSV

The CSV format is probably the most common file format in the context of data analysis and visualization. CSV files contain tabular data that can be viewed and edited in a spreadsheet software such as Excel. CSV stands for [comma-separated values](https://en.wikipedia.org/wiki/Comma-separated_values), which seems to say it all: the data values are separated by commas and each row represents one item. However, there are also CSV files that use separators other than commas, such as tabs and semicolons. 

Let's load a CSV file! Thankfully Pandas has the convenient `read_csv()` method ready for us, which can open CSV data via a file path or URL, and turns it directly into a DataFrame object. 

Here we're retrieving information about [childcare places in Potsdam](https://opendata.potsdam.de/explore/dataset/kitaboerse-20161108/export/) from Potsdam's Open Data portal. This file happens to be in a CSV format that uses the semicolon as a separator (hence `sep=";"` as the second parameter in the method call):

In [ ]:
kitas = pd.read_csv(
  "https://opendata.potsdam.de/explore/dataset/kitaboerse-20161108/download/",
  sep=";")

kitas.head()

,name_der_kindertagesbetreuungseinrichtung,stand_vom,barrierefreie_einrichtung,kinderkrippe_0_3_j,tagespflege_0_3_j,padagogisch_begleitete_spielgruppe_0_3_j,kindergarten_3_j_schuleintritt,hort_ab_schuleintritt,andere_kinderbetreuung_ab_3_klasse,strasse,hausnummer,postleitzahl,kartenansicht,tel_nr,fax_nr,e_mail,homepage_der_einrichtung,stadtteil,trager,tel_nr_trager,e_mail_trager,homepage_des_tragers,platze_unbefristet,darunter_betriebskita_unbefristet,befristete_betriebserlaubnis_erlaubnis,platze_befristet,darunter_betriebskita_befristet,betrieb_e,befristete_betriebserlaubnis_erlaubnis_gultig_bis_monat,befristete_betriebserlaubnis_erlaubnis_gultig_bis_jahr,nur_tagespflege_erste_hilfe_kurs_gultig_bis_monat,nur_tagespflege_erste_hilfe_kurs_gultig_bis_jahr,integrationseinrichtung,inklusionshort,einrichtung_mit_besonderem_padagogischen_angebot,offnungszeiten_mo_fr,offnungszeiten_ab_6_00_uhr,offnungszeiten_nach_17_30_uhr,wochenendoffnung,abweichende_offnungszeiten,ubernachtung_moglich,schliesstage_von_bis,fruhstuck,mittag,vesper,abendessen,versorgungsart,link_zum_bereich_kinder_und_jugend_der_landeshauptstadt_potsdam,link_zu_anmeldeinformationen_der_einrichtung
0,"Wurzelwerk, AWO, Kita",15.08.2016,Nein,Ja,Nein,Nein,Ja,Nein,Nein,Zum Kahleberg,23A,14478 Potsdam,https://lhp.maps.arcgis.com/apps/webappviewer/...,33.181.718.980,331.817.186,wurzelwerk@awo-potsdam.de,http://www.awo-potsdam.de/einrichtungen-und-di...,Waldstadt II,AWO Kinder- und Jugendhilfe Potsdam gGmbH,0331.58 14 80,info-kjh@awo-potsdam.de,http://www.awo-potsdam.de/awo-bezirksverband/a...,137.0,0.0,Nein,0.0,0.0,NaN,--,NaN,--,NaN,Nein,Nein,Ja,06:30-17:30,Nein,Nein,Nein,NaN,Nein,NaN,Ja,Ja,Ja,Nein,Mischversorgung,http://www.potsdam.de/kita-tipp,http://www.awo-potsdam.de/awo-bezirksverband/a...
1,"Baumhaus, Kita",07.11.2016,Ja,Nein,Nein,Nein,Ja,Ja,Nein,Sternstr.,63,14480 Potsdam,https://lhp.maps.arcgis.com/apps/webappviewer/...,0331.88 74 71 60,0331.88 74 71 61,johannes.hille@independentliving.de,http://www.baumhaus-potsdam.de,Drewitz,IL (Independent Living) Kinder und Jugendhilfe...,0331.201 22 80,irene.seidel@independentliving.de,http://www.independentliving.de/il-kitas-potsdam,128.0,0.0,Nein,0.0,0.0,NaN,--,NaN,--,NaN,Nein,Nein,Ja,6:15 Uhr bis 17:15 Uhr,Nein,Nein,Nein,NaN,Nein,keine,Ja,Ja,Ja,Nein,Mischversorgung,http://www.potsdam.de/kita-tipp,http://www.baumhaus-potsdam.de
2,"Sonnenland, Hort",21.09.2016,Nein,Nein,Nein,Nein,Nein,Ja,Nein,Knobelsdorffstr.,7,14471 Potsdam,https://lhp.maps.arcgis.com/apps/webappviewer/...,0331.95 14 736,0331.95 14 738,hort-sonnenland@ejf.de,https://www.ejf.de/kindertagesstaetten-horte/h...,Potsdam West,EJF - gAG,030.76 88 42 56,jordan-nimsch.sigrid@ejf.de,http://www.ejf.de,135.0,0.0,Nein,0.0,0.0,NaN,--,NaN,--,NaN,Nein,Nein,Nein,6 bis 7. 30 Uhr und 11.30 bis 17 Uhr,Ja,Nein,Nein,Ferien 6 bis 17 Uhr,Nein,NaN,Nein,Ja,Ja,Nein,Eigenversorgung am Standort,http://www.potsdam.de/kita-tipp,https://www.ejf.de/kindertagesstaetten-horte/h...
3,"Pfiffikus, AWO, Kita",08.09.2016,Ja,Ja,Nein,Nein,Ja,Ja,Nein,Pietschkerstr.,44,14480 Potsdam,https://lhp.maps.arcgis.com/apps/webappviewer/...,0331.88 74 95 80,0331.88 74 95 818,pfiffikus@awo-potsdam.de,http://www.awo-potsdam.de/einrichtungen-und-di...,Stern,AWO Kinder- und Jugendhilfe Potsdam gGmbH,0331.58 14 80,info-kjh@awo-potsdam.de,http://www.awo-potsdam.de/awo-bezirksverband/a...,120.0,0.0,Nein,0.0,0.0,NaN,--,NaN,--,NaN,Nein,Nein,Nein,6 bis 18 Uhr,Ja,Nein,Nein,NaN,Nein,NaN,Ja,Ja,Ja,Nein,Eigenversorgung geliefert,http://www.potsdam.de/kita-tipp,http://www.awo-potsdam.de/einrichtungen-und-di...
4,Evangelische Kindertagesstätte Sonnenblume,01.02.2016,Nein,Ja,Nein,Nein,Ja,Nein,Nein,Bellavitestr.,NaN,14480 Potsdam,https://lhp.maps.arcgis.com/apps/webappviewer/...,0331.61 79 90,3.315.056.545,anke.link@hoffbauer-kinder.de,http://www.hoffbauer-bildung.de/kita-sonnenblume,Kirchsteigfeld,Hoffbauer Kinder gGmbH,0331.231 31 00,info@hoffbauer-kinder.de,http://www.hoffbauer-bildung.de,120.0,0.0,Nein,0.0,0.0,NaN,--,NaN,--,Na

✏️ *Try loading another CSV dataset from [Potsdam's Open Data Portal](https://opendata.potsdam.de/)*

### JSON

[JSON](https://en.wikipedia.org/wiki/JSON) is a format that grew in popularity in the context of web development. It stands for JavaScript Object Notation and makes for a compact, yet relatively human-readable format.

Above we have already opened the contents of the local file `anscombe.json` using the `file.read()` method. To parse the contents into the structured form of a DataFrame, with which we can do some analysis, we can rely on Pandas and the handy `read_json()` method. Let's do this with the anscombe data we already opened:

In [ ]:
anscombe = pd.read_json(anscombe_json)
anscombe

,Series,X,Y
0,I,10,8.04
1,I,8,6.95
2,I,13,7.58
3,I,9,8.81
4,I,11,8.33
5,I,14,9.96
6,I,6,7.24
7,I,4,4.26
8,I,12,10.84
9,I,7,4.81


### XML

[XML](https://en.wikipedia.org/wiki/XML) (Extensible Markup Language) is a data format, which can have very different kinds of hierarchical structures. XML files are common in a wide variety of contexts, including libraries, and especially in situations, in which the interoperability of multiple systems by several vendors needs to be ensured.

The 🌲 **ElementTree** module will help us to parse the elements contained in an XML file. 

In [ ]:
import xml.etree.ElementTree as ET

As we have already retrieved the XML feed from Tagesschau (and saved it in the variable `tagesschau_xml`), we can now parse it directly from the string, i.e., using the method `ET.fromstring()`:

In [ ]:
tagesschau = ET.fromstring(tagesschau_xml)
tagesschau

<Element 'rss' at 0x7f3fb82c4048>

This gives the root element of the XML feed (and all its children) in the variable `tagesschau`. 

Going through all items with `findall` and within these with `find` for specific sub-elements, we can extract the publication date and time and the title of the respective item. In the following these elements are put together into the DataFrame `tagesschau_df`. Note that it helps to peek into the XML source of the feed to know the specific element names.

In [ ]:
# create two empty lists
dates = []
titles = []

# go through all item elements in the tree
for item in tagesschau.findall('.//item'):
  # extract date information and titles
  dates.append( item.find('.//pubDate').text )
  titles.append( item.find('.//title').text )

# create a dataframe containing the two columns
tagesschau_df = pd.DataFrame(    
    {'date': dates,
     'title': titles,
    })  

tagesschau_df

,date,title
0,"Tue, 25 Aug 2020 16:03:48 +0200",Lufthansa verzichtet auf Umbuchungsgebühren bi...
1,"Tue, 25 Aug 2020 15:48:32 +0200",Liveblog: ++ Kein Unterrichtsausschluss für Ma...
2,"Tue, 25 Aug 2020 15:46:54 +0200",Streit um Corona-Tests für Reisende
3,"Tue, 25 Aug 2020 15:43:41 +0200",Corona-Proteste: Breite rechtsextreme Mobilisi...
4,"Tue, 25 Aug 2020 15:23:45 +0200",Corona-Infektionen in Spanien: Armee hilft im ...
5,"Tue, 25 Aug 2020 15:09:15 +0200","Maas warnt Türkei und Griechenland vor ""Spiel ..."
6,"Tue, 25 Aug 2020 15:00:51 +0200",Mecklenburg-Vorpommern: Tagestouristen dürfen ...
7,"Tue, 25 Aug 2020 14:52:16 +0200",Corona-Hilfen: Ermittlungen gegen Investitions...
8,"Tue, 25 Aug 2020 14:44:43 +0200",RKI-Studie: Kaum Corona-Infektionen ohne Symptome
9,"Tue, 25 Aug 2020 14:29:54 +0200",Nach Schiffsunglück: Großes Wrackteil vor Maur...


✏️ *Each news item also contains a `description` element. Why not add a third column to the DataFrame?*

### HTML

While not really a data format per se, many datasets are actually published via the document format of the web: [HTML](https://en.wikipedia.org/wiki/HTML) (hypertext markup language). All webpages are in one way or another provided in the form of HTML.

Turning HTML content of webpages into structured data, also known as scraping, is necessary when information is published on webpages, while not (yet) available as structured data sets. For this scenario, the module 🍜 **Beautiful Soup** is your friend! To use it you can import the latest version by referring to `bs4`:


In [ ]:
import bs4

Suppose you want to extract the table of cities on the Wikipedia page [List of largest cities](https://en.wikipedia.org/wiki/List_of_largest_cities) you would first get the content of the page and then involve the magic powers of Beautiful Soup, some of which might look a bit cryptic at this point:


In [ ]:
permalink = "https://en.wikipedia.org/w/index.php?title=List_of_largest_cities&oldid=952676625"
wiki = requests.get(permalink).text

In the variable `wiki` we now have the HTML content of the Wikipedia page. Next we initialize Beautiful Soup's `html.parser` and engage in a bit of data parsing:

In [ ]:
soup = bs4.BeautifulSoup(wiki, "html.parser")

# extract all table elements
tables = soup.find_all('table')

# create some empty lists of city names and populations
names = []
pops = []

for table in tables:
  # iterate over all rows,  <tr> is the html element for table rows
  rows = table.find_all('tr')    
  for row in rows:
    # get all cells per row,  <td> is the html element for table cells
    cells = row.find_all('td')
    if len(cells) > 1:
      # get span elements from cells in population column
      pop = cells[3].find('span')
      # if there is no value, we're omitting this city
      if(pop is None):
        continue
      pops.append(pop.text.replace(",", ""))
      # get name of cities, in first column
      names.append(cells[0].text.replace("\n", ""))

# generate a dataframe based on the city names & populations
df = pd.DataFrame({"city": names, "population": pops})
df.population = df.population.astype(int) # case string to integer values
df

,city,population
0,Tokyo,37400068
1,Delhi,28514000
2,Shanghai,25582000
3,São Paulo,21650000
4,Mexico City,21581000
...,...,...
76,"Washington, D.C.",5207000
77,Yangon,5157000
78,Alexandria,5086000
79,Jinan,5052000


✏️ *Try to extract another column*

## Examining

You are now able to load and parse data from several formats. At this point, here are plenty of ways to inspect these datasets. We are going to try some simple methods to peek around the datasets. Once you have a tabular dataset ready as a DataFrame, there are quite a few convenient methods to view and explore its contents.

### Head & tail

You could start with looking at the beginning of the dataset with `head()`:

In [ ]:
kitas.head()

,name_der_kindertagesbetreuungseinrichtung,stand_vom,barrierefreie_einrichtung,kinderkrippe_0_3_j,tagespflege_0_3_j,padagogisch_begleitete_spielgruppe_0_3_j,kindergarten_3_j_schuleintritt,hort_ab_schuleintritt,andere_kinderbetreuung_ab_3_klasse,strasse,hausnummer,postleitzahl,kartenansicht,tel_nr,fax_nr,e_mail,homepage_der_einrichtung,stadtteil,trager,tel_nr_trager,e_mail_trager,homepage_des_tragers,platze_unbefristet,darunter_betriebskita_unbefristet,befristete_betriebserlaubnis_erlaubnis,platze_befristet,darunter_betriebskita_befristet,betrieb_e,befristete_betriebserlaubnis_erlaubnis_gultig_bis_monat,befristete_betriebserlaubnis_erlaubnis_gultig_bis_jahr,nur_tagespflege_erste_hilfe_kurs_gultig_bis_monat,nur_tagespflege_erste_hilfe_kurs_gultig_bis_jahr,integrationseinrichtung,inklusionshort,einrichtung_mit_besonderem_padagogischen_angebot,offnungszeiten_mo_fr,offnungszeiten_ab_6_00_uhr,offnungszeiten_nach_17_30_uhr,wochenendoffnung,abweichende_offnungszeiten,ubernachtung_moglich,schliesstage_von_bis,fruhstuck,mittag,vesper,abendessen,versorgungsart,link_zum_bereich_kinder_und_jugend_der_landeshauptstadt_potsdam,link_zu_anmeldeinformationen_der_einrichtung
0,"Wurzelwerk, AWO, Kita",15.08.2016,Nein,Ja,Nein,Nein,Ja,Nein,Nein,Zum Kahleberg,23A,14478 Potsdam,https://lhp.maps.arcgis.com/apps/webappviewer/...,33.181.718.980,331.817.186,wurzelwerk@awo-potsdam.de,http://www.awo-potsdam.de/einrichtungen-und-di...,Waldstadt II,AWO Kinder- und Jugendhilfe Potsdam gGmbH,0331.58 14 80,info-kjh@awo-potsdam.de,http://www.awo-potsdam.de/awo-bezirksverband/a...,137.0,0.0,Nein,0.0,0.0,NaN,--,NaN,--,NaN,Nein,Nein,Ja,06:30-17:30,Nein,Nein,Nein,NaN,Nein,NaN,Ja,Ja,Ja,Nein,Mischversorgung,http://www.potsdam.de/kita-tipp,http://www.awo-potsdam.de/awo-bezirksverband/a...
1,"Baumhaus, Kita",07.11.2016,Ja,Nein,Nein,Nein,Ja,Ja,Nein,Sternstr.,63,14480 Potsdam,https://lhp.maps.arcgis.com/apps/webappviewer/...,0331.88 74 71 60,0331.88 74 71 61,johannes.hille@independentliving.de,http://www.baumhaus-potsdam.de,Drewitz,IL (Independent Living) Kinder und Jugendhilfe...,0331.201 22 80,irene.seidel@independentliving.de,http://www.independentliving.de/il-kitas-potsdam,128.0,0.0,Nein,0.0,0.0,NaN,--,NaN,--,NaN,Nein,Nein,Ja,6:15 Uhr bis 17:15 Uhr,Nein,Nein,Nein,NaN,Nein,keine,Ja,Ja,Ja,Nein,Mischversorgung,http://www.potsdam.de/kita-tipp,http://www.baumhaus-potsdam.de
2,"Sonnenland, Hort",21.09.2016,Nein,Nein,Nein,Nein,Nein,Ja,Nein,Knobelsdorffstr.,7,14471 Potsdam,https://lhp.maps.arcgis.com/apps/webappviewer/...,0331.95 14 736,0331.95 14 738,hort-sonnenland@ejf.de,https://www.ejf.de/kindertagesstaetten-horte/h...,Potsdam West,EJF - gAG,030.76 88 42 56,jordan-nimsch.sigrid@ejf.de,http://www.ejf.de,135.0,0.0,Nein,0.0,0.0,NaN,--,NaN,--,NaN,Nein,Nein,Nein,6 bis 7. 30 Uhr und 11.30 bis 17 Uhr,Ja,Nein,Nein,Ferien 6 bis 17 Uhr,Nein,NaN,Nein,Ja,Ja,Nein,Eigenversorgung am Standort,http://www.potsdam.de/kita-tipp,https://www.ejf.de/kindertagesstaetten-horte/h...
3,"Pfiffikus, AWO, Kita",08.09.2016,Ja,Ja,Nein,Nein,Ja,Ja,Nein,Pietschkerstr.,44,14480 Potsdam,https://lhp.maps.arcgis.com/apps/webappviewer/...,0331.88 74 95 80,0331.88 74 95 818,pfiffikus@awo-potsdam.de,http://www.awo-potsdam.de/einrichtungen-und-di...,Stern,AWO Kinder- und Jugendhilfe Potsdam gGmbH,0331.58 14 80,info-kjh@awo-potsdam.de,http://www.awo-potsdam.de/awo-bezirksverband/a...,120.0,0.0,Nein,0.0,0.0,NaN,--,NaN,--,NaN,Nein,Nein,Nein,6 bis 18 Uhr,Ja,Nein,Nein,NaN,Nein,NaN,Ja,Ja,Ja,Nein,Eigenversorgung geliefert,http://www.potsdam.de/kita-tipp,http://www.awo-potsdam.de/einrichtungen-und-di...
4,Evangelische Kindertagesstätte Sonnenblume,01.02.2016,Nein,Ja,Nein,Nein,Ja,Nein,Nein,Bellavitestr.,NaN,14480 Potsdam,https://lhp.maps.arcgis.com/apps/webappviewer/...,0331.61 79 90,3.315.056.545,anke.link@hoffbauer-kinder.de,http://www.hoffbauer-bildung.de/kita-sonnenblume,Kirchsteigfeld,Hoffbauer Kinder gGmbH,0331.231 31 00,info@hoffbauer-kinder.de,http://www.hoffbauer-bildung.de,120.0,0.0,Nein,0.0,0.0,NaN,--,NaN,--,Na

✏️ *What do you think happens, when you replace `head()` with `tail()` ?*

### Describe & info

You can also ask Pandas to provide some statistical descriptions (which are only applied to the columns containing numeric data):


In [ ]:
kitas.describe()

,platze_unbefristet,darunter_betriebskita_unbefristet,platze_befristet,darunter_betriebskita_befristet,befristete_betriebserlaubnis_erlaubnis_gultig_bis_jahr,nur_tagespflege_erste_hilfe_kurs_gultig_bis_jahr
count,136.000000,136.000000,136.000000,135.0,20.000000,1.0
mean,117.727941,1.132353,19.860294,0.0,2016.350000,2015.0
std,81.297302,10.133047,60.033839,0.0,1.225819,NaN
min,0.000000,0.000000,0.000000,0.0,2014.000000,2015.0
25%,54.750000,0.000000,0.000000,0.0,2016.000000,2015.0
50%,105.000000,0.000000,0.000000,0.0,2016.000000,2015.0
75%,161.750000,0.000000,0.000000,0.0,2017.000000,2015.0
max,404.000000,110.000000,269.000000,0.0,2019.000000,2015.0


✏️ *Try any of the other datasets that we parsed above, e.g., `anscombe`*

This may not seem that useful to you yet. You may want to know what kind of datatypes the different columns contain and how many values are present. For this the `info()` method will be of help:

In [ ]:
kitas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 137 entries, 0 to 136
Data columns (total 49 columns):
 #   Column                                                           Non-Null Count  Dtype  
---  ------                                                           --------------  -----  
 0   name_der_kindertagesbetreuungseinrichtung                        137 non-null    object 
 1   stand_vom                                                        137 non-null    object 
 2   barrierefreie_einrichtung                                        137 non-null    object 
 3   kinderkrippe_0_3_j                                               137 non-null    object 
 4   tagespflege_0_3_j                                                137 non-null    object 
 5   padagogisch_begleitete_spielgruppe_0_3_j                         137 non-null    object 
 6   kindergarten_3_j_schuleintritt                                   137 non-null    object 
 7   hort_ab_schuleintritt                       

With this it is now possible to access specific columns by using their names.  But did you notice the long label for the first column? Let's rename the column `name_der_kindertagesbetreuungseinrichtung` into something short and sweet such as: `name`:

In [ ]:
kitas = kitas.rename(columns={"name_der_kindertagesbetreuungseinrichtung": "name"})

✏️ *Do you want to rename any other columns?*

### Select & query

We can select an individual column using single [square brackets]:

In [ ]:
kitas["name"]

0                                  Wurzelwerk, AWO, Kita
1                                         Baumhaus, Kita
2                                       Sonnenland, Hort
3                                   Pfiffikus, AWO, Kita
4             Evangelische Kindertagesstätte Sonnenblume
                             ...                        
132                                Spielgruppe Waldstadt
133                                   Heilig-Kreuz, Kita
134                                   Grasshoppers, Kita
135    Tagespflegepersonen FidL - Frauen in der Leben...
136         Tagespflegepersonen Treffpunkt Fahrland e.V.
Name: name, Length: 137, dtype: object

… and we can select multiple columns using nested [[square brackets]]:

In [ ]:
kitas[["name", "betrieb_e"]]

,name,betrieb_e
0,"Wurzelwerk, AWO, Kita",NaN
1,"Baumhaus, Kita",NaN
2,"Sonnenland, Hort",NaN
3,"Pfiffikus, AWO, Kita",NaN
4,Evangelische Kindertagesstätte Sonnenblume,NaN
...,...,...
132,Spielgruppe Waldstadt,NaN
133,"Heilig-Kreuz, Kita",NaN
134,"Grasshoppers, Kita",NaN
135,Tagespflegepersonen FidL - Frauen in der Leben...,NaN


✏️ *Which columns interest you? Replace `name` and `betrieb_e` with other column labels* 

Note that the `betrieb_e` columns contains a lot of `NaN` - this stands for "Not a Number" and it means here that values are missing.

In order to focus on the rows which do have missing data, we can squeeze in a requirement that we only want those rows, where the values in column `betrieb_e` are not missing, i.e., `notnull()`:

In [ ]:
kitas [kitas["betrieb_e"].notnull()] [["name", "betrieb_e"]]

,name,betrieb_e
38,"Kastanienhof, Hort",davon 37 Plätze am Schulstandort ...
76,"Die Buntstifte, Hort",Röhrenstr. 6 /14480 Potsdam / Galileistr. 6 / ...
81,Kindergarten und Hort der Waldorfschule Potsdam,Kita 50 Plätze - Hort 110 Plätze
96,Betriebskita klEinstein,Studentenwerk Potsdam
107,"Geolino, Kita","GFZ, AWI, PIK"


✏️ *Formulate a query on another column:*

There are four related methods for accessing rows, columns, and specific values, either by integer positons (iloc and iat) or by the labels (that is what is displayed in bold above).

- `loc`: access rows and columns by label
- `iloc`: access rows and columns by integer position 
- `at`: access a single value for a row/column label pair
- `iat`: access a single value for a row/column pair by integer position 

For example, this way we can get the first entry in the `kitas` DataFrame: 

In [ ]:
kitas.loc[0]

# because the index here uses integers, iloc and loc do the same

name                                                                                           Wurzelwerk, AWO, Kita
stand_vom                                                                                                 15.08.2016
barrierefreie_einrichtung                                                                                       Nein
kinderkrippe_0_3_j                                                                                                Ja
tagespflege_0_3_j                                                                                               Nein
padagogisch_begleitete_spielgruppe_0_3_j                                                                        Nein
kindergarten_3_j_schuleintritt                                                                                    Ja
hort_ab_schuleintritt                                                                                           Nein
andere_kinderbetreuung_ab_3_klasse                              

Finally, you can also retrieve rows that match a query. With this we are retrieving the names of the kitas who are operated by AWO:

In [ ]:
kitas[kitas['name'].str.contains('AWO')] ["name"]

0                                  Wurzelwerk, AWO, Kita
3                                   Pfiffikus, AWO, Kita
7                                      Krabbelkäfer, AWO
13                                Turmspatzen, AWO, Kita
14                           Wilde Früchtchen, AWO, Kita
47                       Eltern-Kind-Gruppe im EKIZ, AWO
48                              Am Babelsberg, AWO, Kita
53     Nuthegeister, AWO, Hort für hör-, sprach-, ler...
56                   Integrationskita Sternschnuppe, AWO
61                     Abenteuerland, AWO, Kita und Hort
71                                Sandscholle, AWO, Kita
73                              Tausendfüßler, AWO, Kita
83                                 Inselmäuse, AWO, Kita
102                                Kinderland, AWO, Kita
103                      Kulturhaus Babelsberg, AWO, AKi
113                             Havelsprotten, AWO, Hort
119                    Integrationskita Kinderhafen, AWO
121                            

✏️ *How about all the entries that have "Hort" in their name?*

… or those who have sparrows in their name. Because ‘Spatzen’ is occurring both by itself and as part of compound words, we are querying for uppercase and lowercase versions using the IGNORECASE flag of the regular expression module `re`:

In [ ]:
import re # regular expressions module, use for a case-insensitive query

kitas[kitas['name'].str.contains('Spatzen', flags=re.IGNORECASE, regex=True)] ["name"]

13                                Turmspatzen, AWO, Kita
25                                     Spatzennest, Kita
85     Evangelische integrative Kindertagesstätte Nut...
122                               Spatzenhaus, AWO, Kita
126                              Weberspatzen, AWO, Hort
Name: name, dtype: object

✏️ *Now it's time for you to wrangle data! If you have not done so yet, follow all the pencils and change things around!*

## Sources
- [Pandas Tutorial: DataFrames in Python - DataCamp](https://www.datacamp.com/community/tutorials/pandas-tutorial-dataframe-python)
- [The ElementTree XML API](https://docs.python.org/2/library/xml.etree.elementtree.html)
- [Where do Mayors Come From? Querying Wikidata with Python and SPARQL - Towards Data Science](https://towardsdatascience.com/where-do-mayors-come-from-querying-wikidata-with-python-and-sparql-91f3c0af22e2)
- [External data: Local Files, Drive, Sheets, and Cloud Storage - Colaboratory](https://colab.research.google.com/notebooks/io.ipynb)
- [Loading data: Drive, Sheets, and Google Cloud Storage](https://colab.research.google.com/notebooks/io.ipynb) 
- [Examining Data Using Pandas | Linux Journal](https://www.linuxjournal.com/content/examining-data-using-pandas)
- [Beautiful Soup Documentation](https://www.crummy.com/software/BeautifulSoup/bs4/doc/)